In [3]:
import os
import itertools
import numpy as np
import pandas as pd

In [5]:
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [37, 370, 3441]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'nonlearned'
wandb_project = 'tuned_OXPETS'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=37_random_state=1001_weight_decay=0.01' --n=37 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_OXPETS' --weight_decay=0.01"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=37_random_state=1001_weight_decay=0.001' --n=37 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_OXPETS' --weight_decay=0.001"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hu

In [4]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_acc(df):
    return df.val_or_test_acc.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

def get_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_results(df, experiments_path):
    columns = ['n', 'prior_scale', 'prior_type', 'random_state', 'test_acc', 'test_loss', 'test_nll', 
           'test_prior', 'train_acc', 'train_loss', 'train_nll', 'train_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        if row.prior_scale:
            model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        else:
            model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), row.prior_scale, row.prior_type, int(row.random_state), 
                       last_epoch.val_or_test_acc, last_epoch.val_or_test_loss, 
                       last_epoch.val_or_test_nll, last_epoch.val_or_test_prior, 
                       last_epoch.train_acc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior]
        results.loc[results.shape[0]] = results_row
    return results

In [7]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [370, 3441]
prior_scales = [None]
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0100,370,None,nonlearned,1001,0.689189,0.000010
1,0.0010,370,None,nonlearned,2001,0.635135,0.000001
2,0.0001,370,None,nonlearned,3001,0.554054,0.000000
3,0.0100,3441,None,nonlearned,1001,0.897898,0.001000
4,0.0100,3441,None,nonlearned,2001,0.897898,0.001000
5,0.0100,3441,None,nonlearned,3001,0.911411,0.000001


In [10]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
results = get_results(nonlearned_hyperparameters, experiments_path)
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,370,None,nonlearned,1001,0.575321,1.720191,1.720191,0.0,1.000000,0.000037,0.000037,0.0
1,370,None,nonlearned,2001,0.537177,1.828269,1.828269,0.0,1.000000,0.000633,0.000633,0.0
2,370,None,nonlearned,3001,0.532070,1.782722,1.782722,0.0,0.991892,0.156752,0.156752,0.0
3,3441,None,nonlearned,1001,0.864874,0.555606,0.555606,0.0,1.000000,0.000219,0.000219,0.0
4,3441,None,nonlearned,2001,0.866187,0.547540,0.547540,0.0,1.000000,0.000212,0.000212,0.0
5,3441,None,nonlearned,3001,0.859838,0.655502,0.655502,0.0,1.000000,0.000021,0.000021,0.0


In [11]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc', 'train_acc']
for column in columns:
    grouped_results['{}_mean'.format(column)] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results['{}_std'.format(column)] = grouped_results[column].apply(lambda item: np.std(item))
    grouped_results['{}_min'.format(column)] = grouped_results[column].apply(lambda item: np.min(item))
    grouped_results['{}_max'.format(column)] = grouped_results[column].apply(lambda item: np.max(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,prior_scale,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,train_acc_mean,train_acc_std,train_acc_min,train_acc_max
0,370,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.575320959091, 0.537177324295, 0.532069563866)","(1.72019144146, 1.82826946762, 1.78272174171)","(1.72019144146, 1.82826946762, 1.78272174171)","(0.0, 0.0, 0.0)","(1.0, 1.0, 0.991891860962)","(3.67180605158e-05, 0.0006326181232, 0.1567517...","(3.67180605158e-05, 0.0006326181232, 0.1567517...","(0.0, 0.0, 0.0)",0.548189,0.019298,0.532070,0.575321,0.997297,0.003822,0.991892,1.0
1,3441,nonlearned,"(None, None, None)","(1001, 2001, 3001)","(0.86487364769, 0.866187036037, 0.859837651253)","(0.555605979366, 0.547539691277, 0.655501988781)","(0.555605979366, 0.547539691277, 0.655501988781)","(0.0, 0.0, 0.0)","(1.0, 1.0, 1.0)","(0.000218813053151, 0.000211618911242, 2.14034...","(0.000218813053151, 0.000211618911242, 2.14034...","(0.0, 0.0, 0.0)",0.863633,0.002737,0.859838,0.866187,1.000000,0.000000,1.000000,1.0


In [12]:
grouped_results[['n', 'prior_type', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,n,prior_type,test_acc_mean,test_acc_min,test_acc_max
0,370,nonlearned,0.548189,0.532070,0.575321
1,3441,nonlearned,0.863633,0.859838,0.866187
